In [1]:
import pandas as pd
from datetime import datetime
from urllib.request import HTTPError
from pandas import DataFrame
import numpy as np

In [2]:
# Set up formula for scraping tables from www.basketball-reference.com
base_url = "https://www.basketball-reference.com/leagues/NBA_"
years_list = [*range(2002,2021,1)]
# October usually represented as base url with year but without month, hence the "" as 1st list item
months_list = ["","-november","-december","-january","-february","-march","-april","-may","-june"]

In [3]:
# Create a list of full urls to scrape
urls_list = []
for year in years_list:
    year = year
    for month in months_list:
        month = month
        urls_list.append(f"https://www.basketball-reference.com/leagues/NBA_{year}_games{month}.html")
# print(urls_list)

In [4]:
# Create list of scraped table data from htmls
appended_data = []
for url in urls_list:
    try:
        data = pd.read_html(url)
    except HTTPError:
        print(f"Oops, that url {url} may not exist.")

    # store DataFrames in the list
    appended_data.append(data[0])
# concatenate dataframes to each other as you go
appended_data = pd.concat(appended_data)
appended_data.head()

Oops, that url https://www.basketball-reference.com/leagues/NBA_2012_games-november.html may not exist.
Oops, that url https://www.basketball-reference.com/leagues/NBA_2020_games-may.html may not exist.
Oops, that url https://www.basketball-reference.com/leagues/NBA_2020_games-june.html may not exist.


,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes
0,"Tue, Oct 30, 2001",7:00p,Boston Celtics,108,Cleveland Cavaliers,89,Box Score,NaN,17748,NaN
1,"Tue, Oct 30, 2001",7:30p,Indiana Pacers,97,New Jersey Nets,103,Box Score,NaN,8749,NaN
2,"Tue, Oct 30, 2001",7:30p,Washington Wizards,91,New York Knicks,93,Box Score,NaN,19763,NaN
3,"Tue, Oct 30, 2001",7:30p,Toronto Raptors,85,Orlando Magic,114,Box Score,NaN,16088,NaN
4,"Tue, Oct 30, 2001",8:00p,Philadelphia 76ers,74,Minnesota Timberwolves,83,Box Score,NaN,18697,NaN


In [5]:
# Cleanup, reset index, drop and rename columns
nba_df = appended_data
nba_df = nba_df.reset_index()
nba_df = nba_df.rename(columns={"index":"Game Number","Visitor/Neutral":"Visitor", "PTS":"V_PTS", "Home/Neutral":"Home", "PTS.1":"H_PTS", "Unnamed: 7" :"OT", "Attend.":"Attendence"})
nba_df = nba_df.drop(columns=['Notes','Unnamed: 6'])
nba_df['OT'] = nba_df['OT'].fillna('')
nba_df

,Game Number,Date,Start (ET),Visitor,V_PTS,Home,H_PTS,OT,Attendence
0,0,"Tue, Oct 30, 2001",7:00p,Boston Celtics,108,Cleveland Cavaliers,89,,17748
1,1,"Tue, Oct 30, 2001",7:30p,Indiana Pacers,97,New Jersey Nets,103,,8749
2,2,"Tue, Oct 30, 2001",7:30p,Washington Wizards,91,New York Knicks,93,,19763
3,3,"Tue, Oct 30, 2001",7:30p,Toronto Raptors,85,Orlando Magic,114,,16088
4,4,"Tue, Oct 30, 2001",8:00p,Philadelphia 76ers,74,Minnesota Timberwolves,83,,18697
...,...,...,...,...,...,...,...,...,...
25138,110,"Wed, Apr 15, 2020",7:30p,New Orleans Pelicans,NaN,San Antonio Spurs,NaN,,NaN
25139,111,"Wed, Apr 15, 2020",7:30p,Indiana Pacers,NaN,Washington Wizards,NaN,,NaN
25140,112,"Wed, Apr 15, 2020",10:00p,Los Angeles Lakers,NaN,Phoenix Suns,NaN,,NaN
25141,113,"Wed, Apr 15, 2020",10:00p,Los Angeles Clippers,NaN,Portland Trail Blazers,NaN,,NaN


In [6]:
# Locate and drop "playoffs" banner rows, drop games with no scores (COVID break in 2020), change data types, add datetime
playoff_label = nba_df[nba_df['Date'] == "Playoffs"].index
nba_df = nba_df.drop(playoff_label)
nba_df = nba_df[nba_df['V_PTS'].notna()]
nba_df.astype({'V_PTS': 'int32', 'H_PTS': 'int32', 'Attendence':'int32'}).dtypes
nba_df['Datetime'] = pd.to_datetime(nba_df['Date'], format='%a, %b %d, %Y')
nba_df

,Game Number,Date,Start (ET),Visitor,V_PTS,Home,H_PTS,OT,Attendence,Datetime
0,0,"Tue, Oct 30, 2001",7:00p,Boston Celtics,108,Cleveland Cavaliers,89,,17748,2001-10-30
1,1,"Tue, Oct 30, 2001",7:30p,Indiana Pacers,97,New Jersey Nets,103,,8749,2001-10-30
2,2,"Tue, Oct 30, 2001",7:30p,Washington Wizards,91,New York Knicks,93,,19763,2001-10-30
3,3,"Tue, Oct 30, 2001",7:30p,Toronto Raptors,85,Orlando Magic,114,,16088,2001-10-30
4,4,"Tue, Oct 30, 2001",8:00p,Philadelphia 76ers,74,Minnesota Timberwolves,83,,18697,2001-10-30
...,...,...,...,...,...,...,...,...,...,...
24793,73,"Tue, Mar 10, 2020",10:30p,Los Angeles Clippers,131,Golden State Warriors,107,,18064,2020-03-10
24794,74,"Wed, Mar 11, 2020",7:00p,Detroit Pistons,106,Philadelphia 76ers,124,,20172,2020-03-11
24795,75,"Wed, Mar 11, 2020",7:30p,New York Knicks,136,Atlanta Hawks,131,OT,15393,2020-03-11
24796,76,"Wed, Mar 11, 2020",7:30p,Charlotte Hornets,109,Miami Heat,98,,19600,2020-03-11


In [8]:
# Break up year and month, add season column
nba_df['Datetime'] = pd.to_datetime(nba_df['Datetime'])
nba_df['year'], nba_df['month'] = nba_df['Datetime'].dt.year, nba_df['Datetime'].dt.month
# nba['OT'] = nba['OT'].fillna('')
# game number= game number +1? do i care?
nba_df["Season"] = "hold"

In [11]:
# Label Seasons
y = 2001
for i, column in nba_df.iterrows():
    if (column[10] == y and column[11] > 8) or (column[10] == (y+1) and column[11] < 8):
        season = f'{y}/{y+1}'
        nba_df.at[i,'Season'] = season
    else:
        season = f'{y+1}/{y+2}'
        nba_df.at[i,'Season'] = season
        y = y+1
nba_df

,Game Number,Date,Start (ET),Visitor,V_PTS,Home,H_PTS,OT,Attendence,Datetime,year,month,Season
0,0,"Tue, Oct 30, 2001",7:00p,Boston Celtics,108,Cleveland Cavaliers,89,,17748,2001-10-30,2001,10,2001/2002
1,1,"Tue, Oct 30, 2001",7:30p,Indiana Pacers,97,New Jersey Nets,103,,8749,2001-10-30,2001,10,2001/2002
2,2,"Tue, Oct 30, 2001",7:30p,Washington Wizards,91,New York Knicks,93,,19763,2001-10-30,2001,10,2001/2002
3,3,"Tue, Oct 30, 2001",7:30p,Toronto Raptors,85,Orlando Magic,114,,16088,2001-10-30,2001,10,2001/2002
4,4,"Tue, Oct 30, 2001",8:00p,Philadelphia 76ers,74,Minnesota Timberwolves,83,,18697,2001-10-30,2001,10,2001/2002
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24793,73,"Tue, Mar 10, 2020",10:30p,Los Angeles Clippers,131,Golden State Warriors,107,,18064,2020-03-10,2020,3,2019/2020
24794,74,"Wed, Mar 11, 2020",7:00p,Detroit Pistons,106,Philadelphia 76ers,124,,20172,2020-03-11,2020,3,2019/2020
24795,75,"Wed, Mar 11, 2020",7:30p,New York Knicks,136,Atlanta Hawks,131,OT,15393,2020-03-11,2020,3,2019/2020
24796,76,"Wed, Mar 11, 2020",7:30p,Charlotte Hornets,109,Miami Heat,98,,19600,2020-03-11,2020,3,2019/2020


In [12]:
nba_df.to_csv('nba_final_scores.csv')